# Лабораторна робота 9 (1 частина)

Романенко Ілля ФІТ 4-7

## Крок 1

завантажимо датасет та підготуємо дані

In [1]:
from keras import datasets

(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

2024-10-14 18:49:23.623153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 18:49:23.706232: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 18:49:23.729579: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 18:49:23.876209: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 18:49:25.217087: W tensorflow/compiler/tf2

In [2]:
from keras.utils import to_categorical

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## Крок 2

завантажимо попередньо навчену модель VGG16, без верхніх шарів

In [3]:
from keras.applications import VGG16

base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(32, 32, 3)
)

## Крок 3

заморозимо всі шари окрім останніх 4

In [4]:
for layer in base_model.layers[:-4]:
    layer.trainable = False

додамо зверху 4 власних шари

In [5]:
from keras import layers, models

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [6]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6
)

In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,748,170 (56.26 MB)

 Trainable params: 7,112,906 (27.13 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

## Крок 4

натренуємо модель

In [8]:
history = model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=16, 
    validation_data=(x_test, y_test), 
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/20


2024-10-14 18:49:31.340429: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 769s 246ms/step - accuracy: 0.1349 - loss: 2.2351 - val_accuracy: 0.5418 - val_loss: 1.3008 - learning_rate: 0.0010
Epoch 2/20
2477/3125 ━━━━━━━━━━━━━━━━━━━━ 2:45 256ms/step - accuracy: 0.5367 - loss: 1.3230

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Тестова точність: {test_acc}")